# LAB: Ensambles para clasificación de dígitos

## INTRODUCCION

Vamos a utilizar un dataset de `sklearn` para realizar una clasificación de imágenes de dígitos manuscritos. El objetivo es lograr clasificar a qué dígito pertenece cada una de las imágenes.

Para eso vamos a usar los modelos vistos.

Veamos brevemente la estructura de los datos. Básicamente 

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.DESCR)

In [ ]:
digits.keys()

In [ ]:
digits.images.shape

In [ ]:
digits.data.shape

In [ ]:
digits.target_names.shape

In [ ]:
digits.target.shape

In [ ]:
digits.images[0:2]

In [ ]:
digits.data[0]

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Visualizemos algunos registros del dataset.

In [ ]:
# Generamos los gráficos
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# Ploteamos los dígitos: cada imagen es de 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # etiquetamos la imagen con el target value
    ax.text(0, 7, str(digits.target[i]))

## Random Forest

Podemos usar un clasificador de dígitos muy rápidamente utilizando un Random Forest:

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)

model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

Veamos el reporte de clasificación:

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

Y para evaluarlo, podemos usar la matriz de confusión:

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

Es interesante ver cómo un clasificador basado en Random Forest (sin ningún tipo de tuneo) resulta en una muy precisa clasisicación de los datos de dígitos.

## Extra Tree

Probemos ahora un Extra Tree.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_estimators=1000, class_weight='balanced')

In [ ]:
et.fit(Xtrain, ytrain)
ypred_et = et.predict(Xtest)

In [ ]:
print(metrics.classification_report(ypred_et, ytest))

In [ ]:
mat = confusion_matrix(ytest, ypred_et)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

## Gradient Boosting

¿Qué sucede si usamos un GradientBoosting con los parámetros por defecto?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()

In [ ]:
gb.fit(Xtrain, ytrain)
ypred_gb = gb.predict(Xtest)

In [ ]:
print(metrics.classification_report(ypred_gb, ytest))

In [ ]:
mat = confusion_matrix(ytest, ypred_gb)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

## AdaBoost

Probemos ahora con un AdaBoost con los parámetros por defecto:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier()

In [ ]:
ab.fit(Xtrain, ytrain)
ypred_ab = ab.predict(Xtest)

In [ ]:
print(metrics.classification_report(ypred_ab, ytest))

In [ ]:
mat = confusion_matrix(ytest, ypred_ab)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

Seguramente podrán ver que AdaBoost parece tener una performance notablemente peor que la de los modelos anteriores. Una posibilidad, entonces, es tratar de tunear los hiperparámetros para buscar mejorar su performance.

Tengan en cuenta que la "arquitectura" de AdaBoost es diferente. Unos de sus parámetros es `base_estimator`. Por defecto, `AdaBoostClassifier` utiliza un árbol de decisión (es decir, `base_estimator=DecisionTreeClassifier()`). Por lo cual deben tener en cuenta este hecho al momento de definir la grilla de parámetros para la búsqueda: hay algunos parámetros que corresponden al `base_estimator` -en este caso, árboles- y otros que corresponden a `AdaBoostClassifier`.

**Pista**: pueden pasar parámetros de cada estimador de forma análoga a cómo los pasan en un pipeline: `{'base_estimator__[parametro_del_base_estimator': [grilla]}`

In [ ]:
params = {"base_estimator__max_depth": [1, 10, 50],
          "base_estimator__min_samples_split": [5, 10, 15], 
          "base_estimator__max_features": [5, 25, 61],
          "base_estimator__min_samples_leaf": [5, 10, 15],
          "n_estimators": [100, 500],
          "learning_rate":[0.01, 0.1, 1.0]
         }

grid_ab = RandomizedSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), params, n_iter=150, verbose=1)
grid_ab.fit(Xtrain, ytrain)

In [ ]:
grid_ab.best_score_

In [ ]:
ypred_ab_tuned = grid_ab.predict(Xtest)

In [ ]:
print(metrics.classification_report(ypred_ab_tuned, ytest))

In [ ]:
mat = confusion_matrix(ytest, ypred_ab_tuned)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

## BONUS

Traten de mejorar la performance de Gradient Boosting haciendo un tunning de los parámetros.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

gb = GradientBoostingClassifier()
param_grid_gb = {'n_estimators':[100, 500, 1000] , 
                 'max_depth': [5, 10, 15], 
                 'min_samples_split': [5, 10, 15],
                 'min_samples_leaf': [5, 10, 15],
                 'learning_rate':[0.001, 0.001, 0.1, 1.0]}

In [ ]:
grid = RandomizedSearchCV(gb, param_distributions=param_grid_gb,verbose = 2, n_jobs=3, n_iter=50)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
grid.best_estimator_

In [ ]:
gbbest = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=15,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=15, min_samples_split=15,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

gbbest.fit(Xtrain, ytrain)
grid_pred = gbbest.predict(Xtest)

In [ ]:
print(metrics.classification_report(grid_pred, ytest))